# Print flight parameters from the kmz files

In [ ]:
import zipfile
import os
import xml.etree.ElementTree as ET
import shutil
import sys

def find_kml_file(folder):
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith('.kml'):
                return os.path.join(root, file)
    return None

def extract_kmz(kmz_path, extract_to='temp_kmz'):
    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return find_kml_file(extract_to)

def print_kml_parameters(kml_path):
    ns = {
        'kml': 'http://www.opengis.net/kml/2.2',
        'gx': 'http://www.google.com/kml/ext/2.2'
    }

    tree = ET.parse(kml_path)
    root = tree.getroot()

    print("\n==== 📄 DJI KML Parameters ====\n")

    for elem in root.iter():
        tag = elem.tag.split('}')[-1]  # remove namespace
        text = elem.text.strip() if elem.text else ''
        
        # Only print non-empty values
        if text:
            print(f"{tag}: {text}")

        # If the tag has attributes, show them
        if elem.attrib:
            for attr, val in elem.attrib.items():
                print(f"{tag}[@{attr}]: {val}")

    print("\n==== ✅ Finished ====\n")

def main():
    if len(sys.argv) < 2:
        print("Usage: python parse_dji_kmz.py path_to_file.kmz")
        return

    kmz_file = r"D:\PheNo\_Flight Paths 2025\25-SmartWheatGram-Oblique-20m-75-70.kmz"
    if not kmz_file.lower().endswith(".kmz"):
        print("❌ Please provide a valid .kmz file")
        return

    print(f"📂 Loading KMZ file: {kmz_file}")
    kml_path = extract_kmz(kmz_file)
    if not kml_path:
        print("❌ No .kml file found in the .kmz archive.")
        return

    print_kml_parameters(kml_path)
    print(kml_path)
    shutil.rmtree("temp_kmz")

if __name__ == "__main__":
    main()


# Export flight parameters to csv

In [45]:
import zipfile
import os
import xml.etree.ElementTree as ET
import shutil
import sys
import csv

CSV_FILE = "dji_kmz_data.csv"

def find_kml_file(folder):
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith('.kml'):
                return os.path.join(root, file)
    return None

def extract_kmz(kmz_path, extract_to='temp_kmz'):
    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return find_kml_file(extract_to)

def extract_kml_parameters(kml_path):
    tree = ET.parse(kml_path)
    root = tree.getroot()

    parameters = {}

    for elem in root.iter():
        tag = elem.tag.split('}')[-1]  # remove namespace
        text = elem.text.strip() if elem.text else ''
        if text:
            if tag in parameters:
                parameters[tag] += " | " + text
            else:
                parameters[tag] = text

        for attr, val in elem.attrib.items():
            parameters[f"{tag}[@{attr}]"] = val

    return parameters

def update_csv(parameter_dict, csv_path):
    fieldnames = []

    if os.path.exists(csv_path):
        with open(csv_path, mode='r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            fieldnames = reader.fieldnames if reader.fieldnames else []

    new_keys = set(parameter_dict.keys())
    all_keys = set(fieldnames or [])
    updated_keys = list(sorted(all_keys.union(new_keys)))

    write_header = not os.path.exists(csv_path)

    if not write_header and set(updated_keys) != set(fieldnames):
        with open(csv_path, mode='r', newline='', encoding='utf-8') as old:
            reader = list(csv.DictReader(old))

        with open(csv_path, mode='w', newline='', encoding='utf-8') as f_new:
            writer = csv.DictWriter(f_new, fieldnames=updated_keys)
            writer.writeheader()
            for row in reader:
                writer.writerow(row)
            writer.writerow(parameter_dict)
    else:
        with open(csv_path, mode='a' if not write_header else 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=updated_keys)
            if write_header:
                writer.writeheader()
            writer.writerow(parameter_dict)

def process_folder(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.kmz'):
                try:
                    kmz_file = os.path.join(root, file)
                    folder_name = os.path.basename(root)
                    file_name = os.path.basename(file)

                    print(f"📂 Processing {file_name} from folder {folder_name}")

                    kml_path = extract_kmz(kmz_file)
                    if not kml_path:
                        print(f"❌ No KML found in {file_name}")
                        continue

                    parameters = extract_kml_parameters(kml_path)
                    parameters["FolderName"] = folder_name
                    parameters["FileName"] = file_name
                    print(file_name)
                    update_csv(parameters, CSV_FILE)

                except Exception as e:
                    print(f"❌ Error processing {file}: {e}")
                finally:
                    if os.path.exists("temp_kmz"):
                        shutil.rmtree("temp_kmz")

def main():
    if len(sys.argv) < 2:
        print("Usage: python parse_dji_kmz_folder.py path_to_folder")
        return

    folder_path = sys.argv[1]
    # folder_path = r"D:\PheNo\_Flight Paths 2025\M3M"
    # folder_path = r"D:\PheNo\_Flight Paths 2025\M4T"
    folder_path = r"D:\PheNo\_Flight Paths 2025\M4T"
    if not os.path.isdir(folder_path):
        print("❌ Please provide a valid folder path.")
        return

    process_folder(folder_path)
    print(f"\n✅ All KMZ files processed. Output: {CSV_FILE}")

if __name__ == "__main__":
    main()


📂 Processing 25-DiversityOats-Oblique-20m-75-70.kmz from folder M4T
25-DiversityOats-Oblique-20m-75-70.kmz
📂 Processing 25-DLTFarming-Oblique-20m-75-70.kmz from folder M4T
25-DLTFarming-Oblique-20m-75-70.kmz
📂 Processing 25-G2BFrontiers-Oblique-20m-75-70.kmz from folder M4T
25-G2BFrontiers-Oblique-20m-75-70.kmz
📂 Processing 25-Pilot-Oblique-30m-70-75.kmz from folder M4T
25-Pilot-Oblique-30m-70-75.kmz
📂 Processing 25-ProBarVoll-Oblique-20m-75-70.kmz from folder M4T
25-ProBarVoll-Oblique-20m-75-70.kmz
📂 Processing 25-RobOat-20m-Oblique-80-85.kmz from folder M4T
25-RobOat-20m-Oblique-80-85.kmz
📂 Processing 25-RobOat-30m-Thermal-80-85.kmz from folder M4T
25-RobOat-30m-Thermal-80-85.kmz
📂 Processing 25-SmartWheatBox-Oblique-20m-75-70.kmz from folder M4T
25-SmartWheatBox-Oblique-20m-75-70.kmz
📂 Processing 25-SmartWheatBox-RGB-12m-80-85.kmz from folder M4T
25-SmartWheatBox-RGB-12m-80-85.kmz
📂 Processing 25-SmartWheatBox-Thermal-20m-80-85.kmz from folder M4T
25-SmartWheatBox-Thermal-20m-80-85.

# Ordered columns with data from yaml as well

In [52]:
import zipfile
import os
import xml.etree.ElementTree as ET
import shutil
import sys
import csv
from collections import OrderedDict

CSV_FILE = "dji_kmz_data.csv"

def find_file(folder, filename):
    # Search for a specific file inside extracted folder
    for root, _, files in os.walk(folder):
        for file in files:
            if file.lower() == filename.lower():
                return os.path.join(root, file)
    return None

def extract_kmz(kmz_path, extract_to='temp_kmz'):
    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return extract_to

def extract_parameters_from_xml(xml_path, prefix=""):
    """
    Parse XML and extract all tag texts and attributes into OrderedDict,
    prefix keys to keep KML and WPML separate.
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    parameters = OrderedDict()

    for elem in root.iter():
        tag = elem.tag.split('}')[-1]  # remove namespace
        text = elem.text.strip() if elem.text else ''
        key = f"{prefix}{tag}"
        if text:
            if key in parameters:
                parameters[key] += " | " + text
            else:
                parameters[key] = text

        for attr, val in elem.attrib.items():
            attr_key = f"{key}[@{attr}]"
            parameters[attr_key] = val

    return parameters

def update_csv_preserving_order(parameter_dict, csv_path):
    # FolderName and FileName must be first
    ordered_row = OrderedDict()
    ordered_row["FolderName"] = parameter_dict.pop("FolderName", "")
    ordered_row["FileName"] = parameter_dict.pop("FileName", "")

    for k, v in parameter_dict.items():
        ordered_row[k] = v

    existing_headers = []
    existing_rows = []

    if os.path.exists(csv_path):
        with open(csv_path, mode='r', encoding='utf-8', newline='') as f:
            reader = csv.DictReader(f)
            existing_headers = reader.fieldnames or []
            existing_rows = list(reader)

    final_headers = existing_headers.copy()

    for key in ordered_row:
        if key not in final_headers:
            idx = list(ordered_row.keys()).index(key)
            inserted = False
            for prev_key in reversed(list(ordered_row.keys())[:idx]):
                if prev_key in final_headers:
                    insert_at = final_headers.index(prev_key) + 1
                    final_headers.insert(insert_at, key)
                    inserted = True
                    break
            if not inserted:
                final_headers.append(key)

    with open(csv_path, mode='w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=final_headers)
        writer.writeheader()
        for row in existing_rows:
            full_row = {key: row.get(key, "") for key in final_headers}
            writer.writerow(full_row)
        writer.writerow({key: ordered_row.get(key, "") for key in final_headers})

def process_folder(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.kmz'):
                try:
                    kmz_file = os.path.join(root, file)
                    folder_name = os.path.basename(root)
                    file_name = os.path.basename(file)

                    print(f"📂 Processing {file_name} from folder {folder_name}")

                    extract_folder = extract_kmz(kmz_file)

                    # Extract KML parameters with no prefix
                    kml_path = find_file(extract_folder, "template.kml") or find_file(extract_folder, "flight.kml")
                    kml_params = OrderedDict()
                    if kml_path:
                        kml_params = extract_parameters_from_xml(kml_path, prefix="KML_")
                    else:
                        print(f"⚠️  No KML file found in {file_name}")

                    # Extract WPML parameters with WPML_ prefix
                    wpml_path = find_file(extract_folder, "waylines.wpml")
                    wpml_params = OrderedDict()
                    if wpml_path:
                        wpml_params = extract_parameters_from_xml(wpml_path, prefix="WPML_")
                    else:
                        print(f"⚠️  No waylines.wpml file found in {file_name}")

                    # Combine parameters, plus folder and file info (no prefix)
                    combined_params = OrderedDict()
                    combined_params["FolderName"] = folder_name
                    combined_params["FileName"] = file_name

                    # Insert KML params first (without the "KML_" prefix if you want, but I left prefix for clarity)
                    combined_params.update(kml_params)

                    # Then WPML params
                    combined_params.update(wpml_params)

                    update_csv_preserving_order(combined_params, CSV_FILE)

                except Exception as e:
                    print(f"❌ Error processing {file}: {e}")
                finally:
                    if os.path.exists("temp_kmz"):
                        shutil.rmtree("temp_kmz")

def main():
    if len(sys.argv) < 2:
        print("Usage: python parse_dji_kmz_folder.py path_to_folder")
        return

    folder_path = sys.argv[1]
    folder_path = r"D:\PheNo\_Flight Paths 2025\M3M"
    # folder_path = r"D:\PheNo\_Flight Paths 2025\M4T"
    if not os.path.isdir(folder_path):
        print("❌ Please provide a valid folder path.")
        return

    process_folder(folder_path)
    print(f"\n✅ All KMZ files processed. Output: {CSV_FILE}")

if __name__ == "__main__":
    main()


📂 Processing 25-DiversityOats-MS-30m-85-80.kmz from folder M3M
📂 Processing 25-DLTFarming-MS-20m-85-80.kmz from folder M3M
📂 Processing 25-DLTFarming-Oblique-20m-85-80.kmz from folder M3M
📂 Processing 25-E166-MS-30m-85-80.kmz from folder M3M
📂 Processing 25-E166-Oblique-20m-85-80.kmz from folder M3M
📂 Processing 25-G2BOatFrontiers-MS-20m-85-80.kmz from folder M3M
📂 Processing 25-Pilot-30m-MS-70-75.kmz from folder M3M
📂 Processing 25-ProBarVoll-MS-20m-80-80.kmz from folder M3M
📂 Processing 25-RobOat-12m-MS-85-80.kmz from folder M3M
📂 Processing 25-SmartWheatBox-MS-20m-85-80.kmz from folder M3M
📂 Processing 25-SmartWheatGram-MS-30m-80-80.kmz from folder M3M
📂 Processing 25-Soldeling-MS-30m-80-85.kmz from folder M3M
📂 Processing 25-Søråsjordet-MS-30m-80-80.kmz from folder M3M

✅ All KMZ files processed. Output: dji_kmz_data.csv


In [ ]:
    folder_path = r"D:\PheNo\_Flight Paths 2025\M3M"
    folder_path = r"D:\PheNo\_Flight Paths 2025\M4T"
